# The Battle of Neighborhoods

## Introduction
#### Shake Shack, a burger brand established in New York in 2004 is now very popular. Not only famous in its domestic market, It also has overseas branches in Asia, Middle East, and Britain. However, there is no branches in their nearest neighbor, Canada. This project aimed at comparing New York and Toronto, and look for a suitable area for Shake Shack to open up a new branch.

## Data

#### In this project, we first explore Shake Shack in New York by getting Shake Shack's nearby Venues. Then, we explore nearby Venues in each Borough of Toronto. Finally, we'll try to explore similarities between Shake Shack and each Borough in Toronto.

1. Get New York location data by geopy

In [1]:
import pandas as pd

In [2]:
!pip install geopy
from geopy.geocoders import Nominatim

address = 'New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)



40.7127281 -74.0060152


2. Search Shake Shack in New York

In [3]:
import requests # library to handle requests

CLIENT_ID = '4T3J520CTWULGVMCH2QF4VPKJ2BASI3JPL1SUYECDNBJ5HF0' # your Foursquare ID
CLIENT_SECRET = 'W3N5PZQXWNYTH13HBNRQ32PSBPZHGQSI1QB1S5TC2TPS03RU' # your Foursquare Secret
ACCESS_TOKEN = 'VXJXCGW41ZUOOKB5RFBEKEIWEOHNV2ZGXXP1T4JHOEDSLMAO' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 10000 # define radius

search_query = 'Shake Shack'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format( 
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude,
    ACCESS_TOKEN, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [4]:
from pandas.io.json import json_normalize
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

shake_shack = dataframe_filtered[dataframe_filtered['name'] == 'Shake Shack']
shake_shack

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Shake Shack,Burger Joint,215 Murray St,US,New York,United States,btwn West St & North End Ave,791,"[215 Murray St (btwn West St & North End Ave),...","[{'label': 'display', 'lat': 40.71532572032048...",40.715326,-74.014750,NaN,10282,NY,4dda977ad22d38ef4332c419
1,Shake Shack,Burger Joint,200 Broadway,US,New York,United States,at Fulton St,339,"[200 Broadway (at Fulton St), New York, NY 10038]","[{'label': 'display', 'lat': 40.71070286896417...",40.710703,-74.009024,NaN,10038,NY,5787b68e498efcabbebba4f8
2,Shake Shack,Burger Joint,"E 23rd St and Madison Avenue, Madison Square Park",US,New York,United States,Madison Ave & E 23rd St,3535,"[E 23rd St and Madison Avenue, Madison Square ...","[{'label': 'display', 'lat': 40.74148371088094...",40.741484,-73.988218,Flatiron,10010,NY,40e74880f964a520150a1fe3
4,Shake Shack,Burger Joint,1 Old Fulton St,US,Brooklyn,United States,Water Street,1478,"[1 Old Fulton St (Water Street), Brooklyn, NY ...","[{'label': 'display', 'lat': 40.703053, 'lng':...",40.703053,-73.994004,NaN,11201,NY,538d22cb498e86974754d6da
5,Shake Shack,Burger Joint,409 Fulton St,US,Brooklyn,United States,at Boerum Pl,2721,"[409 Fulton St (at Boerum Pl), Brooklyn, NY 11...","[{'label': 'display', 'lat': 40.69212698255525...",40.692127,-73.988658,NaN,11201,NY,4eeb4c51775b1d4812e0261c
6,Shake Shack,Burger Joint,691 8th Ave,US,New York,United States,at W 44th St,5276,"[691 8th Ave (at W 44th St), New York, NY 10036]","[{'label': 'display', 'lat': 40.75833417228198...",40.758334,-73.988988,Theater District,10036,NY,4c2bccc7d1a10f475a88f864
7,Shake Shack,Burger Joint,152 E 86th St,US,New York,United States,btwn Lexington & 3rd Ave,8563,"[152 E 86th St (btwn Lexington & 3rd Ave), New...","[{'label': 'display', 'lat': 40.77909852079875...",40.779099,-73.954686,NaN,10028,NY,4c1918c7fe5a76b07b0f0315
8,Shake Shack,Burger Joint,"49 Grand Central Terminal, Lower Level Dining ...",US,New York,United States,NaN,5069,"[49 Grand Central Terminal, Lower Level Dining...","[{'label': 'display', 'lat': 40.752651, 'lng':...",40.752651,-73.977108,NaN,10017,NY,51ca35d4498e950b67e87df8
9,Shake Shack,Burger Joint,20 3rd Ave,US,New York,United States,at E 9th St,2364,"[20 3rd Ave (at E 9th St), New York, NY 10003]","[{'label': 'display', 'lat': 40.72999845323382...",40.729998,-73.989696,NaN,10003,NY,59d36de20fe7a024363de0b8
10,Shake Shack,Burger Joint,366 Columbus Ave,US,New York,United States,at W 77th St,7964,"[366 Columbus Ave (at W 77th St), New York, NY...","[{'label': 'display', 'lat': 40.78067635406833...",40.780676,-73.976443,NaN,10024,NY,49b0c608f964a520cd521fe3


3. Visualize Shake Shack in New York

In [5]:
import folium

venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(shake_shack.lat, shake_shack.lng, shake_shack.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

4. Get Shake Shack Nearby Venues

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

shake_shack_venues = getNearbyVenues(names=shake_shack['name'],
                                 latitudes=shake_shack['lat'],
                                 longitudes=shake_shack['lng']
                                )

Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack
Shake Shack


In [7]:
shake_shack_venues.head()

,Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Shake Shack,40.715326,-74.01475,Asphalt Green Battery Park City,40.715924,-74.014673,Athletics & Sports
1,Shake Shack,40.715326,-74.01475,Irish Hunger Memorial,40.715139,-74.016557,Monument / Landmark
2,Shake Shack,40.715326,-74.01475,Teardrop Park,40.716843,-74.015576,Park
3,Shake Shack,40.715326,-74.01475,Shake Shack,40.715326,-74.014750,Burger Joint
4,Shake Shack,40.715326,-74.01475,SoulCycle TriBeCa,40.716178,-74.013007,Cycle Studio


5. Get Toronto Borough Location

In [8]:
from bs4 import BeautifulSoup
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page=requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df=df[['PostalCode', 'Borough', 'Neighborhood']]

locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
toronto_data = df.join(locgeo_df, on='PostalCode')

toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


6. Visualize Toronto

In [9]:
toronto_map = folium.Map(location=[toronto_data['Latitude'].mean(), toronto_data['Longitude'].mean()],
                     zoom_start=12, tiles='Stamen Terrain')

for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=1,
        parse_html=False).add_to(toronto_map)
    
toronto_map

7. Get Toronto Borough Nearby Venues

In [10]:
toronto_venues = getNearbyVenues(names=toronto_data['Borough'],
                                 latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude']
                                )

North York
North York
Downtown Toronto
North York
Queen's Park
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York/East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
W

In [11]:
toronto_venues.head()

,Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
